In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache, SQLiteCache

set_llm_cache(SQLiteCache("cache.db"))

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}
                                     
    {example}
                              
    {start}
"""
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
]


full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts,
)


chain = full_prompt | chat

chain.invoke(
    {
        "character": "Pirate",
        "example_question": "What is your location?",
        "example_answer": "Arrrrg! That is a secret!! Arg arg!!",
        "question": "What is your fav food?",
    }
)

AI: 
I know the following:
Capital: Berlin
Language: German
Food: Bratwurst and Sauerkraut
Currency: Euro

AIMessageChunk(content='AI: \nI know the following:\nCapital: Berlin\nLanguage: German\nFood: Bratwurst and Sauerkraut\nCurrency: Euro')

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatMessagePromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


examples = [
    {
        "movie": "exhuma",
        "answer": """
        Here is what I know:
        Director: Jaehyun Jang
        Main actor: Choi Min-sik, Kim Go-eun, Yoo Hae-jin, Dohyun Lee
        genre: occult, mystery
        synopsis: In Los Angeles, USA, shamans ‘Hwa-rim’ (Kim Go-eun) and ‘Bong-gil’ (Lee Do-hyun) receive a large sum of money. I meet the eldest grandson of a family where a strange disease is passed down from generation to generation. ‘Hwarim’, who realized that his ancestor’s grave site was in trouble, recommended relocating it.‘Sang-deok’ (Choi Min-sik), the best feng shui master who smells money, and undertaker ‘Young-geun’ (Yoo Hae-jin) join the group.“You know everything… “What happens if you touch a grave wrongly?” A strange tomb located in a bad place where people can never be buried.‘Sangdeok’ senses an ominous energy and rejects the offer, but With the persuasion of ‘Hwarim’, the exhumation of the graves eventually begins. Something came out that shouldn't have come out.
        """,
    },
    {
        "movie": "Wonka",
        "answer": """
        I know this:
        Director: King Paul
        Main actor: Timothee Chalamet, Calah Lane, Olivia Colman, Tom Davis, Hugh Grant, Sally Hawkins
        genre: fantasy, drama
        synopsis: The sweetest journey in the world. All good things start with a dream! The dream of wizard and chocolate maker ‘Willy Wonka’. Opening your own chocolate shop in ‘Sweet Department Store’, the mecca of desserts. All I have is a dream full of old hats and only 12 sovereigns. I am confident that I will captivate people with my special magical chocolate. But in a situation where there is nothing to eat, no place to sleep, and no one to rely on, ‘Wonka’, who stayed at an old inn, falls into the scheme of ‘Mrs Scrubbit’ and ‘Bleacher’. With accommodation costs rising like a snowdrift, you quickly fall into debt. In addition, the appearance of ‘Oompa Loompa’, a small thief who steals chocolate every night. There is even a strong check on the chocolate cartel that monopolizes the ‘Sweet Department Store’. The road to becoming the world's best chocolate maker is a difficult one.
        """,
    },
    {
        "movie": "12.12: The Day",
        "answer": """
        I know this:
        Director: KIM Sung-soo
        Main actor: HWANG Jung-min, JUNG Woo-sung, LEE Sung-min, PARK Hae-joon, KIM Sung-kyun
        genre: Souvlaki and Feta Cheese
        synopsis: On December 12, 1979, a military rebellion occurred in the capital Seoul. That day, the fate of Korea changed. After October 26th, which shook the Republic of Korea, a new wind blew in Seoul for a while. On December 12, Security Commander Chun Du-gwang revolted. Mobilize all private organizations within the military and bring even front-line troops to Seoul. Chun Doo-gwang's rebels, who are blinded by power, and Lee Tae-sin, the commander of the capital guard, stand against them. Between the suppression forces, 9 hours pass by at a moment's notice. A tense confrontation between two forces, risking their lives. Tonight, the fiercest war unfolds in the capital of South Korea!
        """,
    },
]


example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What do you know about {movie}?"),
        ("ai", "{answer}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a movie expert, you give short answers."),
        example_prompt,
        ("human", "What do you know about {movie}?"),
    ]
)

chain = final_prompt | chat

chain.invoke({"movie": "Cinderella"})

I know this:
        Director: Kay Cannon
        Main actor: Camila Cabello, Billy Porter, Idina Menzel, Nicholas Galitzine, Pierce Brosnan, Minnie Driver
        genre: musical, fantasy
        synopsis: A modern musical with a bold take on the classic fairy tale. Our ambitious heroine has big dreams and with the help of her Fab G, she perseveres to make them come true.

AIMessageChunk(content='I know this:\n        Director: Kay Cannon\n        Main actor: Camila Cabello, Billy Porter, Idina Menzel, Nicholas Galitzine, Pierce Brosnan, Minnie Driver\n        genre: musical, fantasy\n        synopsis: A modern musical with a bold take on the classic fairy tale. Our ambitious heroine has big dreams and with the help of her Fab G, she perseveres to make them come true.')